## Please don't run / don't click "Run all" the notebook:
At the time of writing of this document, the current core limit is 200 cores per workspace and depending upon number of concurrent users, you may end up with core capacity being exceeded or maximum number of parallel jobs being exceeded error.         


## Scenario Overview

The demo scenario is of anomaly detection in manufacturing. We have developed an end-to-end manufacturing scenario using **telemetry sensors data** from 1000 machines running in the manufacturing plant. We develop an anomaly detection model to identify instances or data points that deviate heavily from the normal data and mark these abnormal or suspicious events in data. These deviated points are referred to as anomalies or outliers.         

The anomaly detection model is developed using the ** Scikit-learn IsolationForest** algorithm. There are 1000 machines of four different models. Each machine contains four components of interest, and four sensors measuring voltage, pressure, vibration, and rotation in real-time.



**Note:** This notebook is written using PySpark and scikit-learn.

**Steps:** 

1) Data is ingested from Azure Data Lake Storage Gen2 (ADLS Gen2) using PySpark.

2) The anomaly detection model is built using 
   IsolationForest library.
   
3) This model is used to identify instances or data points that deviate heavily from the normal data and mark these abnormal or suspicious events in data


## Data Ingestion


***setting-up pyspark environment***


In [4]:
# Load the libraries
from pyspark.sql import SparkSession

# Create the pyspark environment
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

***Declaring datafile URLs and table names***


In [5]:
# Define urls for all data
DATA_BASE_URI='abfss://mfgdemodata@#DATA_LAKE_NAME#.dfs.core.windows.net/'
MACHINES_URI= DATA_BASE_URI + "machines.csv"
TELEMETRY_URI= DATA_BASE_URI + "telemetryp.csv"

# Define hive table names for all data
MACHINES_TABLE = "machines_table"
TELEMETRY_TABLE="telemetry_table"
ANOMALY_DATA_TABLE="anomaly_data_table"



***Machines dataset*** 

+ Simulated data tracking 1000 machines (under 4 different models)  
+ It is an asset-level data giving static information on machines   
+ As we can find in data exploration, 'model 3' has the highest number of machines  
+ Attributes: machineID, model, age(service age in years)

In [4]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(MACHINES_TABLE)):
   machine_spark = spark.read.load(MACHINES_URI, format='csv',header=True,inferschema=True)
   machine_spark.write.saveAsTable(MACHINES_TABLE)

# Display machine dataset
display(spark.table(MACHINES_TABLE))

***Telemetry dataset***

+ Simulated sensor measurement data from machines in real-time from each of the 1000 machines 
+ the simulated data is an aggregated data averaged over a one hour 
+ telemetry dataset is the largest dataset here, with close to 9 million rows   
+ Attributes: datetime, machineID, volt, rotate, pressure, vibration  

In [5]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(TELEMETRY_TABLE)):
   telemetry_spark = spark.read.load(TELEMETRY_URI, format='csv',header=True,inferschema=True)
   telemetry_spark.write.saveAsTable(TELEMETRY_TABLE)

# Display machine dataset
display(spark.table(TELEMETRY_TABLE))

## Exploratory Data Analysis (EDA) 


*** Converting spark dataframes into pandas***


In [8]:
## Getting pandas datframes for data visualization
machines_pd = spark.table(MACHINES_TABLE).toPandas()
telemetry_pd = spark.table(TELEMETRY_TABLE).toPandas()

/home/trusted-service-user/cluster-env/env/lib/python3.6/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
  warnings.warn("pyarrow.open_stream is deprecated, please use "

### Importing libraries for data visualization 


In [7]:
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd

*** Visualizing machines dataset***


### Matplotlib Plot


In [9]:
# sorting dataframe by models 
machines_pd = machines_pd.sort_values(by='model') 

# matplotlib styling 
plt.style.use('seaborn-whitegrid')
# contoliing plot size 
fig = plt.figure(figsize=(12,4))
# matplotlib histogram
plt.hist(machines_pd['model'], color = 'steelblue', edgecolor = 'steelblue') 
plt.xlabel('Models')
plt.ylabel('Number of Machines')
plt.show()

In [10]:
# matplotlib styling 
plt.style.use('seaborn-whitegrid')
# contoliing plot size 
fig = plt.figure(figsize=(12,6))

# stacked histograms of models by age 
plt.hist([machines_pd.loc[machines_pd['model'] == 'model1', 'age'],
         machines_pd.loc[machines_pd['model'] == 'model2', 'age'],
         machines_pd.loc[machines_pd['model'] == 'model3', 'age'],
         machines_pd.loc[machines_pd['model'] == 'model4', 'age']],
         20,
         stacked = True,
         color= ['skyblue', 'olive', 'bisque', 'lightseagreen'],
         edgecolor='dimgrey',
         label=['model 1', 'model 2', 'model 3', 'model 4'])
plt.legend() 
plt.xlabel('Service Age (yrs)')
plt.ylabel('Number of Machines')
plt.show()

***Visualizing telemetry dataset*** 

Telemetry data is huge and we won’t plot for all the machines. However, for representation, we’ll randomly select one machine and then we will select a time window of 1 month. We will then plot temperature, $volt$,  $rotate$, $pressure$,  and $vibration$ for this sliced data. 


### Seaborn Plot

In [11]:
# format datetime field which comes in as string
telemetry_pd['datetime'] = pd.to_datetime(telemetry_pd['datetime'], format="%Y-%m-%d %H:%M:%S") 

# randomly slicing data with a single machine, machineID = "123"    
telemetry_plot_data = telemetry_pd.loc[telemetry_pd['machineID'] == 123] 
# further slicing data by date range  
telemetry_plot_data = telemetry_plot_data.loc[(telemetry_plot_data['datetime'] >= pd.to_datetime('2015-02-01')) &
                       (telemetry_plot_data['datetime'] <= pd.to_datetime('2015-03-01'))]


# seaborn styling 
sns.set_style("whitegrid")
# contoliing plot size 

# fig, axs = plt.subplots(nrows = 4)
fig, axs = plt.subplots(4,1, figsize=(14,6))

# plotting voltages 
sns.lineplot(x = 'datetime', y ='volt', color = 'red', linewidth = 0.5,  data = telemetry_plot_data, ax=axs[0] )
sns.lineplot(x = 'datetime', y ='rotate', color = 'blue', linewidth = 0.5,  data = telemetry_plot_data, ax=axs[1])
sns.lineplot(x = 'datetime', y ='pressure', color = 'green', linewidth = 0.5,  data = telemetry_plot_data, ax=axs[2]) 
sns.lineplot(x = 'datetime', y ='vibration', color = 'cyan', linewidth = 0.5,  data = telemetry_plot_data, ax=axs[3])
plt.show()

## Feature Engineering


We have conducted a high-level data exploration with the machines and telemetry data. We will now merge these datasets and then transform the merged dataset to create training data for our outlier detection model.
We will use the scikit learn library for anomaly detection.






In [12]:
## Setup our environment by importing required libraries
import pyspark.sql.functions as F
import pyspark

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, round
from pyspark.sql.functions import datediff
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()

Measurements such as temperature, pressure, vibration, etc. are not likely to change in every observation, however, their average values over a long duration of period, such as 12 hours might be better for capturing the variations that help in help in detecting outlier behaviors.


***Telemetry Features***

The telemetry data has 8761000 hourly observations for out 1000 machines. We will aggregate average sensor measures on a tumbling 12-hour window.



In [13]:
telemetry = spark.table(TELEMETRY_TABLE).cache()
# rolling mean and standard deviation
# Temporary storage for rolling means
tel_mean = telemetry

# Which features are we interested in telemetry data set
rolling_features = ['volt','rotate', 'pressure', 'vibration']
      
# n hours = n * 3600 seconds  
time_val = 12 * 3600

# Choose the time_val hour timestamps to align the data
# dt_truncated looks at the column named "datetime" in the current data set.
dt_truncated = ((round(unix_timestamp(col("datetime")) / time_val) * time_val).cast("timestamp"))

The following code blocks alignes the data on 12 hour observations and calculates a rolling mean of the telemetry data over the last 12 hour lags.

In [14]:
# We choose windows for our rolling windows 12hrs
lags = [12]

# align the data
for lag_n in lags:
    wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        tel_mean = tel_mean.withColumn(col_name+'_rollingmean_'+str(lag_n), 
                                       F.avg(col(col_name)).over(wSpec))
        tel_mean = tel_mean.withColumn(col_name+'_rollingstd_'+str(lag_n), 
                                       F.stddev(col(col_name)).over(wSpec))

# Calculate lag values...
telemetry_feat = (tel_mean.withColumn("dt_truncated", dt_truncated)
                  .drop('volt', 'rotate', 'pressure', 'vibration')
                  .fillna(0)
                  .groupBy("machineID","dt_truncated")
                  .agg(F.mean('volt_rollingmean_12').alias('volt_rollingmean_12'),
                       F.mean('rotate_rollingmean_12').alias('rotate_rollingmean_12'), 
                       F.mean('pressure_rollingmean_12').alias('pressure_rollingmean_12'), 
                       F.mean('vibration_rollingmean_12').alias('vibration_rollingmean_12'), )).cache()

In [15]:
telemetry_feat.show(5)

+---------+-------------------+-------------------+---------------------+-----------------------+------------------------+
|machineID|       dt_truncated|volt_rollingmean_12|rotate_rollingmean_12|pressure_rollingmean_12|vibration_rollingmean_12|
+---------+-------------------+-------------------+---------------------+-----------------------+------------------------+
|      243|2015-01-13 00:00:00|  169.5876951645235|   383.95760834919946|      98.57636833480592|       41.24439049978698|
|      540|2015-01-13 12:00:00| 171.68636726140232|    463.5089561787179|      99.72133089761682|       40.78386567090468|
|      737|2015-01-13 00:00:00|  168.8733862248128|   426.01124080940144|     100.97277350773527|      37.539873912583495|
|      737|2015-01-14 00:00:00| 172.32207064127485|    442.1646565094755|      97.37223586017103|       41.50059169322157|
|      243|2015-05-21 00:00:00| 173.78418872987197|    450.3884591647556|      99.01354129062777|        43.0244373067699|
+---------+-----

***Machine Features***

Machine data will be used as is. However,  we will create a set of dummy variables to create to indicate the model of the machine.

In [16]:
machines_feat = spark.table(MACHINES_TABLE).cache()

In [17]:
# join telemetry with machine features to create final feature matrix

# now join that with machines features
final_feat = (telemetry_feat.join(machines_feat, 
                                     ((telemetry_feat['machineID'] == machines_feat['machineID'])), "left")).drop(telemetry_feat.machineID)

In [18]:
final_feat.show(5)

+-------------------+-------------------+---------------------+-----------------------+------------------------+---------+------+---+
|       dt_truncated|volt_rollingmean_12|rotate_rollingmean_12|pressure_rollingmean_12|vibration_rollingmean_12|machineID| model|age|
+-------------------+-------------------+---------------------+-----------------------+------------------------+---------+------+---+
|2015-01-13 00:00:00|  169.5876951645235|   383.95760834919946|      98.57636833480592|       41.24439049978698|      243|model4| 16|
|2015-01-13 12:00:00| 171.68636726140232|    463.5089561787179|      99.72133089761682|       40.78386567090468|      540|model3| 10|
|2015-01-13 00:00:00|  168.8733862248128|   426.01124080940144|     100.97277350773527|      37.539873912583495|      737|model3| 10|
|2015-01-14 00:00:00| 172.32207064127485|    442.1646565094755|      97.37223586017103|       41.50059169322157|      737|model3| 10|
|2015-05-21 00:00:00| 173.78418872987197|    450.3884591647556

In [19]:
# Download the data and save the dataframe into Hive table if not present
if not(spark.catalog._jcatalog.tableExists(ANOMALY_DATA_TABLE)):
   final_feat.write.saveAsTable(ANOMALY_DATA_TABLE)

# Display machine dataset
display(spark.table(ANOMALY_DATA_TABLE))

##### Anomaly Detection

In this section we are using **IsolationForest** algorithm to predict the anomaly and anomaly score and for each data point. We will train our algorithms with entire dataset to detect outliers. This approach is also referred to as outlier detection.


In [6]:
# Load the libraries
import pandas as pd


# Convert the spark dataframe into pandas dataframe
pd_dataframe=spark.table(ANOMALY_DATA_TABLE).toPandas()

In [7]:
## setting up the credentials for ADLS Gen2
import os
from azure.storage.blob import BlockBlobService

# setting up blob storage configs
STORAGE_ACCOUNT_NAME = "#DATA_LAKE_NAME#"
STORAGE_ACCOUNT_ACCESS_KEY = "#STORAGE_ACCOUNT_KEY#"
STORAGE_CONTAINER_NAME = "azureml-mfg"

blob_service = BlockBlobService(STORAGE_ACCOUNT_NAME, STORAGE_ACCOUNT_ACCESS_KEY) 

# Saving the final dataframe to csv 
pd_dataframe.to_csv('mfg_anomaly_pdm.csv',index=False)

output_file_path=os.path.join(os.getcwd(), "mfg_anomaly_pdm.csv")
output_blob_file= "mfg_anomaly_pdm.csv"

# uploading the csv to  the ADLSGen2 storage container
blob_service.create_blob_from_path(STORAGE_CONTAINER_NAME,output_blob_file ,output_file_path)